In [1]:
from pyspark.sql import SparkSession
from pathlib import Path

# Setup sys.path to import from src/
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))



spark = (
    SparkSession.builder
        .appName("ChunkPreprocessor")
        .config("spark.executor.memory", "8g") 
        .getOrCreate()
)


In [2]:
from src.preprocessing import load_data, clean_data, feature_engineer, assemble_features

In [10]:
# Load raw dataset
file_path = "../data/streamingData/streamingData.csv"
raw_df = load_data(file_path, spark)

+--------------------+---------+-------------+-------------+--------------------+--------------------+------------------+----------------+-------------+-----------------+--------------+
|        user_session|num_views|num_cart_adds|num_purchases|       session_start|         session_end|         avg_price|session_duration|main_category|unique_categories|label_category|
+--------------------+---------+-------------+-------------+--------------------+--------------------+------------------+----------------+-------------+-----------------+--------------+
|000081ea-9376-4eb...|        1|            1|            1|2019-10-24T05:05:...|2019-10-24T05:08:...|            131.51|             194|  electronics|                1|   electronics|
|000174ac-0ea3-402...|        6|            1|            2|2019-10-18T06:44:...|2019-10-18T06:46:...| 241.2244444444445|             115|  electronics|                1|   electronics|
|0002b07c-85cd-46e...|       18|            2|            2|2019-10-14

In [6]:
# Clean the dataset
clean_df = clean_data(raw_df)

# Quick check after cleaning
# clean_df.show(5)
clean_df.count()

42448762

In [7]:
feature_df = feature_engineer(clean_df, True)

# feature_df.show(5)
feature_df.count()

483608

In [8]:
feature_df.coalesce(1).write.mode("overwrite").option("header", True).csv("../data/streamingData")